In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.storage import LocalFileStore

loader = UnstructuredFileLoader("document.txt")
cache_dir = LocalFileStore(f".cache")
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    separators=["\n\n", ".", "?", "!"],
    chunk_size=1000,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,  cache_dir)

vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriever = vectorstore.as_retriever()

llm = ChatOpenAI(temperature=0.1,
                 model="gpt-4o-mini",
                 )

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key = "history",
    llm = llm,
)

prompt = ChatPromptTemplate.from_messages([    
    ("system", 
     "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}"),
    MessagesPlaceholder(variable_name="history")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": RunnableLambda(load_memory), 
    }
    | prompt
    | llm
)

In [23]:
def invoke_chain(question):
    result = chain.invoke(question).content
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print(result)

In [24]:
invoke_chain("Who is Julia?")

Julia is a character who is significant to Winston, the protagonist. She is someone he loves and has a deep emotional connection with, but their relationship is complicated by the oppressive regime they live under. Julia is portrayed as a rebellious figure who initially shares Winston's desire for freedom, but their experiences in the Party's system lead to feelings of betrayal between them.
